<a href="https://colab.research.google.com/github/akashban812/DS-practice/blob/main/Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os as os
os.chdir('/content/drive/MyDrive/DS notebooks')
os.getcwd()
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


/content/drive/MyDrive/DS notebooks


In [7]:
#@title Default title text
##Importing the data
##Unicode escape in activated to encounter any error arising due to any invalid characters

## Using the 'Online Retail' dataset from Kaggle


rtl_raw_data=pd.read_csv('OnlineRetail.csv',encoding='unicode_escape')



In [13]:
###checking distribution by country
country_cust=rtl_raw_data[['Country','CustomerID']].drop_duplicates()
country_cust_grp= country_cust.groupby(['Country'])['CustomerID'].nunique().reset_index().sort_values(by=['CustomerID'],ascending=[False])
country_cust_grp

##In the next step we'll use only the records for UK as it has the max customers


,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [16]:
rtl_raw_UK=rtl_raw_data[rtl_raw_data['Country']=='United Kingdom'].reset_index(drop=True)
rtl_raw_UK

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
495473,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,09-12-2011 12:31,1.95,15804.0,United Kingdom
495474,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,09-12-2011 12:49,2.95,13113.0,United Kingdom
495475,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,09-12-2011 12:49,1.25,13113.0,United Kingdom
495476,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,09-12-2011 12:49,8.95,13113.0,United Kingdom
